In [5]:
class connector():
    def __init__(self, name=None):
        self._name = name
        self._value = None
        self._constraints = []
        self._informant = None
        
    def _notify(self, source, message):
        notify = {
            'new_val': lambda constraint : constraint.new_val(),
            'forget': lambda constraint : constraint.forget(),
        }
        for constraint in self._constraints:
            if source != constraint:
                notify[message](constraint)
                
    @property
    def val(self):
        return self._value
    
    @property
    def has_val(self):
        return self._value is not None
    
    def set_val(self, source, val):
        if self._value is None:
            self._informant, self._value = source, val
            if self._name is not None:
                print(self._name, '=', val)
            self._notify(source, 'new_val')
        else:
            if self._value != val:
                print('Contradiction detected:', self._value, 'vs', val)
                
    def connect(self, constraint):
        self._constraints.append(constraint)
        
    def forget(self, source):
        if self._informant == source:
            self._informant, self._value = None, None
            self._notify(source, 'forget')
        
class ternary_constraint():
    def __init__(self, a, b, c, ab, ca, cb):
        self._a = a
        self._b = b
        self._c = c
        self._ab = ab
        self._ca = ca
        self._cb = cb
        for connector in (self._a, self._b, self._c):
            connector.connect(self)
        
    def new_val(self):
        av, bv, cv = [connector.has_val for connector in (self._a, self._b, self._c)]
        if av and bv:
            self._c.set_val(self, self._ab(self._a.val, self._b.val))
        elif av and cv:
            self._b.set_val(self, self._ca(self._c.val, self._a.val))
        elif bv and cv:
            self._a.set_val(self, self._cb(self._c.val, self._b.val))
#         print("not satisfied")
        
    def forget(self):
        for connector in (self._a, self._b, self._c):
            connector.forget(self)
            
def adder(a, b, c):
    from operator import add, sub
    return ternary_constraint(a, b, c, add, sub, sub)

def multiplier(a, b, c):
    from operator import mul, truediv
    return ternary_constraint(a, b, c, mul, truediv, truediv)

def constant(connector, value):
    constraint = {}
    connector.set_val(constraint, value)
    return constraint

def make_converter(c, f):
    """Connect c to f with constraints to convert from Celsius to Fahrenheit."""
    u, v, w, x, y = [connector() for _ in range(5)]
    multiplier(c, w, u)
    multiplier(v, x, u)
    adder(v, y, f)
    constant(w, 9)
    constant(x, 5)
    constant(y, 32)

celsius = connector('Celsius')
fahrenheit = connector('Fahrenheit')
make_converter(celsius, fahrenheit)

celsius.set_val('c', 25)

Celsius = 25
Fahrenheit = 77.0
